In [22]:
import numpy as np
import sympy as sp

In [23]:
def productoria(points, step, txt=False, **kwargs):
    x =  kwargs['x'] if 'x' in kwargs else points[step][0]
    evaluado = 1
    noEvaluado = ""
    for i in range(step):
        noEvaluado += f'*(x-{points[i][0]})'
        evaluado *= (x - points[i][0])
    return evaluado if not txt else noEvaluado   

In [24]:
def alphas(points:list):
    alphaList = [] 
    
    def prevAlphas(p):
        acum = 0
        for i in range(len(alphaList)):
            acum += alphaList[i] if i < 1 else alphaList[i]*productoria(points,i,x=p)
        return acum

    for i,p in enumerate(points):           
        alpha = (points[i][1]-(prevAlphas(p[0]) if i>0 else 0))/productoria(points, i) 
        alphaList.append(alpha)
       
    return alphaList

In [25]:
def nEquation(points, **kwargs):
    pn =  kwargs['pn'] if 'pn' in kwargs else len(points)
    x = sp.symbols('x')
    equations = []
    alphaList = alphas(points)
    equation = f'{alphaList[0]}'

    for i in range(pn):
        if i > 0:
            equation += f'{"+" if alphaList[i] > 0 else ""}{alphaList[i]}{productoria(points,i,True)}'

    equation = sp.expand(equation)
    return sp.lambdify(x,equation,np)

In [26]:
def validarNum(txt:str, nat=False, **kwargs):
    min =  kwargs['min'] if 'min' in kwargs else None
    while True:
        val = input(txt)  
        try:
            val = float(val)
            if val.is_integer():
                val = int(val)
            if (min and val < min):
                raise Exception(f"Error: El número debe ser mayor de {min}") 
            if (nat and (val <= 0)):
                raise Exception("Error: Se esperaba un número mayor a 0")                
            return val
        except ValueError:
            print("Error: Se esperaba un valor numérico válido.") 
        except Exception as e:
            print(e)             

In [27]:
def choice(txt:str):
    ch = str(input(txt))
    while(ch.lower() != 's' and ch.lower() != 'n'):
        print(f'Se esperaba S/N pero se obtubo {ch}.')
        ch = str(input(txt))
    return True if ch.lower() == 's' else False

In [42]:
class Puntos:
    def __init__(self, manualStep=False):
        self.steps = int(validarNum('Ingrese la cantidad de puntos:',True)) if manualStep else 20
        self.XN = np.array([])
        self.YN = np.array([])
    
    def generar(self):
        for i in range(self.steps):
            self.XN = np.append(self.XN, validarNum(f'Ingrese el valor para X{i}'))
            self.YN = np.append(self.YN, validarNum(f'Ingrese el valor para Y{i}'))

    def obtenerPuntos(self):
        self.generar()
        return [(x,y) for x,y in zip(self.XN, self.YN)]

class PuntosAleatorios(Puntos):
    def __init__(self, X_Equal_Y, manualStep=False):
        Puntos.__init__(self, manualStep)
        self.x_min = validarNum('Ingrese la cota menor para x:')
        self.x_max = validarNum('Ingrese la cota mayor para x:',min=self.x_min)
        self.y_min = self.x_min
        self.y_max = self.x_max
    
        if not X_Equal_Y:
            self.y_min = validarNum('Ingrese la cota inferior para y:')
            self.y_max = validarNum('Ingrese la cota superior para y:',min=self.y_min)
         
    def generar(self):
        self.XN = np.random.uniform(self.x_min, self.x_max, self.steps)
        self.YN = np.random.uniform(self.y_min, self.y_max, self.steps)

In [43]:
def main():
    try:
        lista_puntos = Puntos().obtenerPuntos() if choice("Desea ingresar los puntos manualmente? S/N") else PuntosAleatorios(choice('Limites de x e y iguales? S/N')).obtenerPuntos()
        lista_puntos_asc = sorted(lista_puntos, key=lambda x: x[0])
        lista_puntos_desc = sorted(lista_puntos, key=lambda x: x[0], reverse=True)
        print(lista_puntos)
    except Exception:
        print('La creación de la lista de puntos falló.')

[(3.8497290268680113, 2.618527547475191), (1.0926387194185776, 2.0387553377983334), (3.7095325476103183, 3.7635536211321066), (1.7579572166563193, 2.054320827660498), (4.2499872705353665, 2.031169140440916)]
